In [1]:
# import dependencies
import pandas as pd

# import pyarrow as pa
# import pyarrow.parquet as pq

In [2]:
# import glob

# # Get a list of all CSV files in a directory
# csv_files = glob.glob('data/*.csv')

# # Create an empty list to store the dataframes
# dfs = []

# # Loop through the CSV files and read them into Pandas dataframes
# for file in csv_files:
#     df = pd.read_csv(file,encoding='latin-1')
#     dfs.append(df)


In [3]:
# csv_files

In [4]:
#Loading data
df1 = pd.read_csv("data/KJV_Bible.csv")
df2 =pd.read_csv("data/ASV_Bible.csv")
df3 =pd.read_csv("data/t_bbe.csv")
df4 =pd.read_csv("data/t_dby.csv",encoding='latin-1')
df5 =pd.read_csv("data/t_web.csv")
df6 =pd.read_csv("data/YLT_Bible.csv")
df7 =pd.read_csv("data/t_wbt.csv")
df8 = pd.read_csv('data/cross_references.txt',delimiter = "\t",low_memory=False)

In [5]:
vs1 = pd.read_csv("resources/bible_version_key.csv")
# vs = vs1.to_parquet('resources/bible_version_key.parquet',engine='fastparquet')

In [6]:
# Removing unnecessary columns
df1 = df1.drop(['Unnamed: 0'],axis=1)
df1.head()

,id,book,chapter,verse,text,author
0,1001001,Genesis,1,1,In the beginning God created the heaven and th...,Moses
1,1001002,Genesis,1,2,"And the earth was without form, and void; and ...",Moses
2,1001003,Genesis,1,3,"And God said, Let there be light: and there wa...",Moses
3,1001004,Genesis,1,4,"And God saw the light, that it was good: and G...",Moses
4,1001005,Genesis,1,5,"And God called the light Day, and the darkness...",Moses


In [7]:
# setting kjv to df1
kjv_df = df1

In [8]:
#Renaming bible books
kjv_df['book'] = kjv_df['book'].replace(['1 Samuel (1 Kings)','2 Samuel (2 Kings)','1 Kings (3 Kings)','2 Kings (4 Kings)',
                                'Song of Solomon (Canticles)'],['1 Samuel','2 Samuel','1 Kings','2 Kings','Song of Solomon'])

In [9]:
# Creating a function to clean the data
def clean(df):
    dt = {'b':list(range(1, 67)),
        'book': kjv_df['book'].unique().tolist()}
    dt_df = pd.DataFrame(dt)
    df = pd.merge(df,dt_df,how='left',on='b')
    df = df[['id','book','c','v','t']]
    df = df.rename(columns={'c':'chapter','v':'verse','t':'text'})
    return df

In [10]:
# Applying the function to all bible version
asv_df = clean(df=df2)
bbe_df = clean(df=df3)
dby_df =clean(df=df4)
web_df = clean(df=df5)
ylt_df =clean(df=df6)
wbt_df =clean(df=df7)

In [11]:
# Cleaning cross reference data
df8 =pd.read_csv('data/cross_references.txt',delimiter = "\t",low_memory=False)
df8 = df8.rename(columns={'To Verse':'ToVerse'})
df8.shape

(343609, 4)

In [12]:
# Further cleaning of cross reference data. i.e breaking out cross reference with extended verses
df_08 = (df8.set_index(df8.columns.drop('ToVerse',1).tolist()).ToVerse.str.split('-', 
            expand=True).stack().reset_index().rename(columns={0:'ToVerse'}).loc[:, df8.columns])
df_08.shape

(431543, 4)

In [13]:
# Further cleaning of cross references
votes = df_08[['Votes']]
base = df_08['From Verse'].str.split(".", expand = True)
base = base.rename(columns={0:'Abv',1:'chapter',2:'verse'})
# base = pd.concat([base,votes],axis=1)
ref =df_08['ToVerse'].str.split(".", expand = True)
ref = ref.rename(columns={0:'Abv',1:'chapter',2:'verse'})
ref = pd.concat([ref,votes],axis=1)

In [14]:
ref.shape

(431543, 4)

In [15]:
base.shape

(431543, 3)

In [16]:
# creating a dataframe to join abbreviation to all the translations
data = {'Abv':base.Abv.unique().tolist(),
        'book': df1.book.unique().tolist()}
abv_df = pd.DataFrame(data)

In [17]:
# Creating a merging functions for all the translations
def abbrevation(k):
    k = pd.merge(k,abv_df,on='book',how='left')
    return k

In [18]:
abv_k = abbrevation(k=kjv_df)
abv_a = abbrevation(k=asv_df)
abv_b = abbrevation(k=bbe_df)
abv_d = abbrevation(k=dby_df)
abv_w = abbrevation(k=web_df)
abv_y =abbrevation(k=ylt_df)
abv_wb = abbrevation(k=wbt_df)

In [19]:
abv_k['version'] = 'KJV'
abv_a['version'] = 'ASV'
abv_b['version'] = 'BBE'
abv_d['version'] = 'DARBY'
abv_w['version'] = 'WEB'
abv_y['version'] = 'YLT'
abv_wb['version'] = 'WBT'

In [20]:
tranl = abv_k['version'].values[0]
tranl

'KJV'

In [21]:
# # write the clean data to csv file
# abv_k.to_csv('resources/kjv.csv')
# abv_a.to_csv('resources/asv.csv')
# abv_b.to_csv('resources/bbe.csv')
# abv_d.to_csv('resources/dby.csv')
# abv_w.to_csv('resources/web.csv')
# abv_y.to_csv('resources/ylt.csv')
# abv_wb.to_csv('resources/wbt.csv')

In [22]:
# abv_k.to_parquet('resources/kjv.parquet',engine='fastparquet')
# abv_a.to_parquet('resources/asv.parquet',engine='fastparquet')
# abv_b.to_parquet('resources/bbe.parquet',engine='fastparquet')
# abv_d.to_parquet('resources/dby.parquet',engine='fastparquet')
# abv_w.to_parquet('resources/web.parquet',engine='fastparquet')
# abv_y.to_parquet('resources/ylt.parquet',engine='fastparquet')
# abv_wb.to_parquet('resources/wbt.parquet',engine='fastparquet')

In [23]:
# pd.read_parquet('resources/kjv.parquet')

In [24]:
# Creating a function that would merge text to base cross reference for all translations
def base_merge(p):
    base[['chapter','verse']] = base[['chapter','verse']].astype(int)
    p= pd.merge(base,p,on=['Abv','chapter','verse'],how='left')
    p = p[['id','Abv','book','chapter','verse','text']]
    return p

In [25]:
# Applying merge functions to all translation for base scripture references
base_k = base_merge(p=abv_k)
base_a = base_merge(p=abv_a)
base_b = base_merge(p=abv_b)
base_d = base_merge(p=abv_d)
base_w = base_merge(p=abv_w)
base_y =base_merge(p=abv_y)
base_wb = base_merge(p=abv_wb)

In [26]:
# # Creating a merging functions for all cross references scriptures
# def ref_merge(q):
#     ref[['chapter','verse']] = ref[['chapter','verse']].astype(int)
#     q= pd.merge(ref,q,on=['Abv','chapter','verse'],how='left')
#     q = q.rename(columns={'Abv':'Abv_ref','book':'book_ref','chapter':'chapter_ref','verse':'verse_ref','text':'text_ref'})
#     q = q[['id','Abv_ref','book_ref','chapter_ref','verse_ref','text_ref','Votes']]
#     return q

In [27]:
# Creating a merging functions for all cross references scriptures
def ref_merge(q):
    ref[['chapter','verse']] = ref[['chapter','verse']].astype(int)
    q= pd.merge(ref,q,on=['Abv','chapter','verse'],how='left')
    q = q.rename(columns={'id':'ref_id','Abv':'ref_Abv','book':'ref_book','chapter':'ref_chapter','verse':'ref_verse','text':'ref_text'})
    q = q[['ref_id','ref_Abv','ref_book','ref_chapter','ref_verse','ref_text','Votes']]
    return q

In [28]:
# Applying merging function to all cross references
ref_k = ref_merge(q=abv_k)
ref_a = ref_merge(q=abv_a)
ref_b = ref_merge(q=abv_b)
ref_d = ref_merge(q=abv_d)
ref_w = ref_merge(q=abv_w)
ref_y =ref_merge(q=abv_y)
ref_wb = ref_merge(q=abv_wb)

In [29]:
ref_wb.shape

(431596, 7)

In [30]:
# translations complete references
kjv_ref = pd.concat([base_k,ref_k],axis=1)
asv_ref = pd.concat([base_a,ref_a],axis=1)
bbe_ref = pd.concat([base_b,ref_b],axis=1)
dby_ref = pd.concat([base_d,ref_d],axis=1)
web_ref = pd.concat([base_w,ref_w],axis=1)
ylt_ref = pd.concat([base_y,ref_y],axis=1)
wbt_ref = pd.concat([base_wb,ref_wb],axis=1)

In [31]:
kjv_ref

,id,Abv,book,chapter,verse,text,ref_id,ref_Abv,ref_book,ref_chapter,ref_verse,ref_text,Votes
0,1001001,Gen,Genesis,1,1,In the beginning God created the heaven and th...,23065017,Isa,Isaiah,65,17,"For, behold, I create new heavens and a new ea...",6
1,1001001,Gen,Genesis,1,1,In the beginning God created the heaven and th...,19033006,Ps,Psalms,33,6,By the word of the LORD were the heavens made;...,12
2,1001001,Gen,Genesis,1,1,In the beginning God created the heaven and th...,18038004,Job,Job,38,4,Where wast thou when I laid the foundations of...,35
3,1001001,Gen,Genesis,1,1,In the beginning God created the heaven and th...,66022013,Rev,Revelation,22,13,"I am Alpha and Omega, the beginning and the en...",3
4,1001001,Gen,Genesis,1,1,In the beginning God created the heaven and th...,66010006,Rev,Revelation,10,6,And sware by him that liveth for ever and ever...,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431538,66022021,Rev,Revelation,22,21,The grace of our Lord Jesus Christ be with you...,45016020,Rom,Romans,16,20,And the God of peace shall bruise Satan under ...,3
431539,66022021,Rev,Revelation,22,21,The grace of our Lord Jesus Christ be with you...,53003018,2Thess,2 Thessalonians,3,18,The grace of our Lord Jesus Christ be with you...,4
431540,66022021,Rev,Revelation,22,21,The grace of our Lord Jesus Christ be with you...,49006023,Eph,Ephesians,6,23,"Peace be to the brethren, and love with faith,...",1
431541,66022021,Rev,Revelation,22,21,The grace of our Lord Jesus Christ be with you...,49006024,Eph,Ephesians,6,24,Grace be with all them that love our Lord Jesu...,1


In [32]:
# kjv_ref.to_parquet('resources/kjv_ref.parquet',engine='pyarrow')

In [33]:
# # Writing cross reference to csv
# kjv_ref.to_parquet('resources/kjv_ref.parquet',engine='fastparquet')
# asv_ref.to_parquet('resources/asv_ref.parquet',engine='fastparquet')
# bbe_ref.to_parquet('resources/bbe_ref.parquet',engine='fastparquet')
# dby_ref.to_parquet('resources/dby_ref.parquet',engine='fastparquet')
# web_ref.to_parquet('resources/web_ref.parquet',engine='fastparquet')
# ylt_ref.to_parquet('resources/ylt_ref.parquet',engine='fastparquet')
# wbt_ref.to_parquet('resources/wbt_ref.parquet',engine='fastparquet')

In [34]:
# # Writing cross reference to csv
# kjv_ref.to_csv('resources/kjv_ref.csv')
# asv_ref.to_csv('resources/asv_ref.csv')
# bbe_ref.to_csv('resources/bbe_ref.csv')
# dby_ref.to_csv('resources/dby_ref.csv')
# web_ref.to_csv('resources/web_ref.csv')
# ylt_ref.to_csv('resources/ylt_ref.csv')
# wbt_ref.to_csv('resources/wbt_ref.csv')

In [35]:
# chapters = df[df['book'] == book_name]['chapter'].unique()
#     for chapter in chapters:
        

In [36]:
# for book in abv_k['book']:
#     for chapter in abv_k['chapter']:
#         abv_k['chapter_texts'] = " ".join(abv_k[(abv_k['book'] == book) & (abv_k['chapter'] == chapter)]["text"].tolist())

In [37]:
def text_concatenation(df):
    # Creating a multi-index groupby object based on 'book' and 'chapter'
    grouped = df.groupby(['book', 'chapter'])
    
    # Applying a join operation to concatenate texts within each group
    chapter_texts = grouped['text'].apply(lambda texts: ' '.join(texts))

    # Resetting index to turn the multi-index series into a DataFrame
    result_df = chapter_texts.reset_index(name='chapter_texts')

    return result_df

In [38]:
# Example usage of the function with the abv_k DataFrame
abk_c = text_concatenation(abv_k)
asv_c = text_concatenation(abv_a)
bbe_c = text_concatenation(abv_b)
dby_c = text_concatenation(abv_d)
web_c = text_concatenation(abv_w)
ylt_c = text_concatenation(abv_y)
wbt_c = text_concatenation(abv_wb)

In [39]:
abv_kc = pd.merge(abv_k, abk_c, on =['book', 'chapter'], how='left')
abv_ac = pd.merge(abv_a, asv_c, on =['book', 'chapter'], how='left')
abv_cc = pd.merge(abv_b, bbe_c, on =['book', 'chapter'], how='left')
abv_dc = pd.merge(abv_d, dby_c, on =['book', 'chapter'], how='left')
abv_wc = pd.merge(abv_w, web_c, on =['book', 'chapter'], how='left')
abv_yc = pd.merge(abv_y, ylt_c, on =['book', 'chapter'], how='left')
abv_wbc = pd.merge(abv_wb, wbt_c, on =['book', 'chapter'], how='left')

In [40]:
abv_wbc

,id,book,chapter,verse,text,Abv,version,chapter_texts
0,1001001,Genesis,1,1,In the beginning God created the heaven and th...,Gen,WBT,In the beginning God created the heaven and th...
1,1001002,Genesis,1,2,"And the earth was without form, and void; and ...",Gen,WBT,In the beginning God created the heaven and th...
2,1001003,Genesis,1,3,"And God said, Let there be light: and there wa...",Gen,WBT,In the beginning God created the heaven and th...
3,1001004,Genesis,1,4,"And God saw the light, that it was good: and G...",Gen,WBT,In the beginning God created the heaven and th...
4,1001005,Genesis,1,5,"And God called the light Day, and the darkness...",Gen,WBT,In the beginning God created the heaven and th...
...,...,...,...,...,...,...,...,...
31101,66022017,Revelation,22,17,"And the Spirit and the bride say, Come. And le...",Rev,WBT,And he showed me a pure river of water of life...
31102,66022018,Revelation,22,18,For I testify to every man that heareth the wo...,Rev,WBT,And he showed me a pure river of water of life...
31103,66022019,Revelation,22,19,And if any man shall take away from the words ...,Rev,WBT,And he showed me a pure river of water of life...
31104,66022020,Revelation,22,20,"He who testifieth these things saith, Surely I...",Rev,WBT,And he showed me a pure river of water of life...


In [41]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
def summarize_with_t5(text, max_length=50):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# summary = summarize_with_t5(your_text)


In [45]:
abv_kc['chapter_summary'] = abv_kc['chapter_texts'].apply(lambda x: summarize_with_t5(x))

KeyboardInterrupt: 

In [48]:
import numpy as np
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def batch_summarize_with_t5(texts, batch_size=10, max_length=50):
    summaries = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_inputs = tokenizer.batch_encode_plus(["summarize: " + text for text in batch_texts], 
                                                   return_tensors="tf", 
                                                   max_length=512, 
                                                   padding=True, 
                                                   truncation=True)

        summary_ids = model.generate(batch_inputs['input_ids'], 
                                     max_length=max_length, 
                                     min_length=5, 
                                     length_penalty=2.0, 
                                     num_beams=4, 
                                     early_stopping=True)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
    return summaries

# Assuming abv_kc is your DataFrame and 'chapter_texts' is the column with texts



RuntimeError: Failed to import transformers.models.t5.modeling_tf_t5 because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'typeDict'

In [49]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Ensure the model is in evaluation mode
model.eval()

# If you have a GPU, put the model on the GPU
if torch.cuda.is_available():
    model = model.cuda()

def batch_summarize_with_t5(texts, batch_size=10, max_length=50):
    summaries = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_inputs = tokenizer.batch_encode_plus(batch_texts, return_tensors="pt", 
                                                   max_length=512, padding=True, truncation=True)

        # If you have a GPU, send the inputs to the GPU
        if torch.cuda.is_available():
            batch_inputs = {k: v.cuda() for k, v in batch_inputs.items()}

        with torch.no_grad():  # Disable gradient calculation for inference
            summary_ids = model.generate(**batch_inputs, 
                                         max_length=max_length, 
                                         min_length=5, 
                                         length_penalty=2.0, 
                                         num_beams=4, 
                                         early_stopping=True)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
    return summaries


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [50]:
abv_kc['chapter_summary'] = batch_summarize_with_t5(abv_kc['chapter_texts'].tolist())

KeyboardInterrupt: 

In [42]:
docx

NameError: name 'docx' is not defined

In [ ]:
#importing libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
   
# Input text - to summarize 
text = docx
   
# Tokenizing the text
stopWords = set(stopwords.words("english"))
words = word_tokenize(text)
   
# Creating a frequency table to keep the 
# score of each word
   
freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1
        
#creating a dictionary to keep the score
# of each sentence
sentences = sent_tokenize(text)
sentenceValue = dict()
   
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
   
   

sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

#Average value of a sentence from the original text
   
average = int(sumValues / len(sentenceValue))
   
# Storing sentences into our summary.
summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
        summary += " " + sentence
print(summary)

In [ ]:
from sumy.summarizers.lsa import LsaSummarizer

In [ ]:
# Parsing the text string using PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
parser=PlaintextParser.from_string(docx,Tokenizer('english'))

In [ ]:
# creating the summarizer
lsa_summarizer=LsaSummarizer()
lsa_summary= lsa_summarizer(parser.document,10)

# Printing the summary
for sentence in lsa_summary:
    print(sentence)

In [ ]:
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [ ]:
# Creating a summary of 3 sentences.
my_parser = PlaintextParser.from_string(docx,Tokenizer('english'))
lex_rank_summarizer = LexRankSummarizer()
lexrank_summary = lex_rank_summarizer(my_parser.document,sentences_count=10)

# Printing the summary
for sentence in lexrank_summary:
    print(sentence)

In [ ]:
import nltk
import heapq

def summarize(text, num_sentences=5):
    nltk.download("punkt")  # Download the required NLTK resource

    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)

    # Compute the frequency of each word in the text
    word_frequencies = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word not in word_frequencies.keys():
                word_frequencies[word] = 0
            word_frequencies[word] += 1

    # Compute the weight of each sentence by summing the frequency of its words
    sentence_weights = {}
    for sentence in sentences:
        sentence_weight = 0
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_frequencies.keys():
                sentence_weight += word_frequencies[word]
        sentence_weights[sentence] = sentence_weight

    # Extract the top `num_sentences` sentences with the highest weights
    summarized_sentences = heapq.nlargest(num_sentences, sentence_weights, key=sentence_weights.get)

    # Join the extracted sentences into a single string
    summary = " ".join(summarized_sentences)

    return summary


In [ ]:
nltk??

In [ ]:
summarize(docx)

In [ ]:
from fuzzywuzzy import fuzz

def fuzzy_match(text, phrase, threshold=70):
    score = fuzz.token_set_ratio(text, phrase)
    return score >= threshold

text = docx
phrase = "This is the bone of my bone"

if fuzzy_match(text, phrase):
    print("Fuzzy match found!")
else:
    print("Fuzzy match not found.")

In [ ]:
def load_bible(resources):
    df = pd.read_csv(resources)
    return df
bibles = ["kjv", "dby", "asv", "bbe", "web", "wbt", "ylt"]
refs = ["kjv_ref", "dby_ref", "asv_ref", "bbe_ref", "web_ref", "wbt_ref", "ylt_ref"]

for bible, ref in zip(bibles, refs):
    locals()[bible] = load_bible(f"resources/{bible}.csv")
    locals()[ref] = load_bible(f"resources/{ref}.csv")

vs = load_bible("resources/bible_version_key.csv")

In [ ]:
locals()[bible]

In [ ]:
!pip install glob3